In [2]:
import scipy.stats as sps
import numpy as np

V = 14
precision = 6
N = 100

import numpy as np
import csv

def export_table(arr, file_name):
    precision = 6

    # Write data to a CSV file
    with open(f"{file_name}.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        for row in arr:
            row = [f"{np.round(x, precision - 1)}" for x in row]
            writer.writerow(row)


In [9]:
#(D|M|1|0)
def task1():
  t_demand = 0.803
  mu = 1.239
  l = 1 #processing demand number
  l_j = 1 #current demand number
  l_print = 1 #printable num of demand
  k = 1 #quantity of served demands
  t_event = t_demand #moment of event execution
  current_event_stat = 1 #demand is recieved and is serving
  current_sys_stat = 1 #system is serving a demand
  np.random.seed(12345)
  times_of_services = sps.expon.rvs(scale=1/mu, size=N)
  remaining_serv_time = times_of_services[0]
  remaining_demand_time = t_demand
  table_1 = []
  table_2 = [None] * N
  table_2[0] = [1, l * t_demand, times_of_services[k - 1], l * t_demand + times_of_services[k - 1]]
  table_3 = [[0,0,0,0], [0,0,0,0]]
  for j in range(2, 102):
    table_1.append([j-1, t_event, current_event_stat, current_sys_stat, remaining_serv_time, remaining_demand_time, l_print])
    #time to serve less then time to recieve new demand
    if remaining_serv_time < remaining_demand_time and current_sys_stat == 1:
      t_event = t_event + remaining_serv_time
      current_event_stat = 3
      current_sys_stat = 0
      remaining_demand_time -= remaining_serv_time
      remaining_serv_time = -1
      l_print = l

    elif remaining_serv_time > remaining_demand_time:
      t_event += remaining_demand_time
      current_event_stat = 2
      current_sys_stat = 1
      remaining_serv_time -= remaining_demand_time
      remaining_demand_time = t_demand
      l_j += 1
      l_print = l_j
      table_2[l_j - 1] = [l_j, t_event, 0, t_event]

    elif current_sys_stat == 0:
      if j != 101:
        table_3[0][2] += remaining_demand_time #time of waiting
      t_event += remaining_demand_time
      current_event_stat = 1
      current_sys_stat = 1
      remaining_serv_time = times_of_services[k]
      remaining_demand_time = t_demand
      l_j += 1
      l = l_j
      l_print = l_j
      k += 1
      table_2[l - 1] = [l, l * t_demand, times_of_services[k - 1], l * t_demand + times_of_services[k - 1]]

  max_demand_num = 0
  for arr in table_1:
    if arr[-1] > max_demand_num:
      max_demand_num = arr[-1]
  table_2 = table_2[0:max_demand_num]

  for row in table_1:
    table_3[row[3]][0] += 1
  table_3[0][2] += t_demand
  table_3[1][2] = table_1[-1][1] - table_3[0][2]

  table_3[0][1] = table_3[0][0]/(table_3[0][0] + table_3[1][0])
  table_3[1][1] = table_3[1][0]/(table_3[0][0] + table_3[1][0])

  table_3[0][3] = table_3[0][2]/(table_3[0][2] + table_3[1][2])
  table_3[1][3] = table_3[1][2]/(table_3[0][2] + table_3[1][2])

  return table_1, table_2, table_3

t1, t2, t3 = task1()
t3[0].insert(0, 0)
t3[1].insert(0, 1)
export_table(t1, "table_11")
export_table(t2, "table_12")
export_table(t3, "table_13")

print(f"Count of demands: {len(t2)}")
print(f"Count of unserved demands: {sum(map((lambda x: x[2] == 0), t2))}")


Count of demands: 67
Count of unserved demands: 33


In [10]:
#(M|D|1|0)
def task2():
  t_serv = 0.796
  e_lambda = 1.053
  l = 1 #processing demand number
  l_j = 1 #current demand number
  l_print = 1 #printable num of demand
  k = 1 #quantity of served demands
  current_event_stat = 1 #demand is recieved and is serving
  current_sys_stat = 1 #system is serving a demand
  np.random.seed(2048)
  times_of_demandes = sps.expon.rvs(scale=1/e_lambda, size=N)
  remaining_demand_time = times_of_demandes[1]
  remaining_serv_time = t_serv
  t_event = times_of_demandes[0] #moment of event execution
  q = [1]
  task = 1
  table_1 = []
  table_2 = [None] * N
  table_2[0] = [1, t_event, 0, 0, t_event, t_serv, t_event + t_serv]
  table_3 = [[0, 0, 0, 0]]
  table_3[0][2] = t_event

  for j in range(2, 102):
    table_1.append([j-1, t_event, current_event_stat, current_sys_stat, remaining_serv_time, remaining_demand_time, l_print])
    #time to serve less then time to recieve new demand
    if remaining_serv_time <= remaining_demand_time and current_sys_stat > 0:
      if j != 101:
        if len(table_3) < current_sys_stat+1:
          table_3.append([0, 0, 0, 0])
        table_3[current_sys_stat][2] += remaining_serv_time

      t_event = t_event + remaining_serv_time
      current_event_stat = 2
      current_sys_stat -= 1
      task = q.pop(0)
      l_print = task
      t_serv_start = t_event - t_serv
      table_2[task - 1][3] = t_serv_start - table_2[task - 1][1]
      if table_2[task - 1][3] < 0.000001:
        table_2[task - 1][3] = 0
      table_2[task - 1][4] = t_serv_start
      table_2[task - 1][5] = t_serv
      table_2[task - 1][6] = t_event

      remaining_demand_time -= remaining_serv_time
      remaining_serv_time = t_serv

    elif remaining_serv_time > remaining_demand_time or current_sys_stat == 0:
      if j != 101:
        if len(table_3) < current_sys_stat+1:
          table_3.append([0, 0, 0, 0])
        table_3[current_sys_stat][2] += remaining_demand_time

      t_event += remaining_demand_time
      current_event_stat = 1
      if current_sys_stat == 0:
        remaining_serv_time = t_serv
      else:
        remaining_serv_time -= remaining_demand_time
      current_sys_stat += 1
      l_j += 1
      remaining_demand_time = times_of_demandes[l_j]
      q.append(l_j)
      l_print = l_j
      table_2[l_j - 1] = [l_j, t_event, current_sys_stat-1, -1, -1, -1, -1]

  max_demand_num = 0
  for arr in table_1:
    if arr[-1] > max_demand_num:
      max_demand_num = arr[-1]
  table_2 = table_2[0:max_demand_num]


  #if table_1[-1][2] == 1
  for row in table_1:
    table_3[row[3]][0] += 1

  for i in range(len(table_3)):
    table_3[i][1] = table_3[i][0] / 100
    table_3[i][3] = table_3[i][2] / table_1[-1][1]


  return table_1, table_2, table_3
t1, t2, t3 = task2()

for i in range(len(t3)):
    t3[i].insert(0, i)
export_table(t1, "table_21")
export_table(t2, "table_22")
export_table(t3, "table_23")


print(f"Count of demands: {len(t2)}")
print(f"Count of unserved demands: {sum(map((lambda x: x[3] != -1), t2))}")
z = 0
for i in range(100):
    cnt = 0
    t_event = t1[i][1]
    for j in range(len(t2)):
        if t2[j][1] >= t_event:
            cnt += 1
    z += cnt
print(f"z(100): {z/100}")

print(f"Count of demands: {len(t2)}")
serv_demands = sum(map((lambda x: x[2] == 2), t1))
print(f"Count of served demands: {serv_demands}")
z = 0
for i in range(100):
    cnt = 0
    t_event = t1[i][1]
    for j in range(len(t2)):
        if t2[j][1] >= t_event:
            cnt += 1
    z += cnt
print(f"z(100): {z/100}")

sum_q_time = 0
sum_stay_time = 0
for i in range(serv_demands):
    sum_q_time += t2[i][3]
    sum_stay_time += (t2[i][6] - t2[i][1])
q_time_mean = sum_q_time / serv_demands
print(f"q_time_mean: {round(q_time_mean, 5)}")
stay_time_mean = sum_stay_time / serv_demands
print(f"stay_time_mean: {round(stay_time_mean, 5)}")
print(f"downtime: {round(t3[0][3] / t1[-1][1], 5)}")


Count of demands: 52
Count of unserved demands: 48
z(100): 25.62
Count of demands: 52
Count of served demands: 48
z(100): 25.62
q_time_mean: 1.75044
stay_time_mean: 2.54644
downtime: 0.12091


In [11]:
#(M|M|1|0)
def task3():
  mu = 1.239
  e_lambda = 1.053
  l = 1 # processing demand number
  l_j = 1 # current demand number
  l_print = 1 # printable num of demand
  current_event_stat = 1 # demand is recieved and is serving
  current_sys_stat = 1 # system is serving a demand
  np.random.seed(2048)
  times_of_demandes = sps.expon.rvs(scale=1/e_lambda, size=N)
  remaining_demand_time = times_of_demandes[1]
  times_of_services = sps.expon.rvs(scale=1/mu, size=N)
  remaining_serv_time = times_of_services[0]
  t_event = times_of_demandes[0] # moment of event execution
  q = [1]
  task = 1
  table_1 = []
  table_2 = [None] * N
  table_2[0] = [1, t_event, 0, -1, -1, -1, -1]
  table_3 = [[0, 0, 0, 0]]
  table_3[0][2] = t_event

  for j in range(2, 102):
    table_1.append([j-1, t_event, current_event_stat, current_sys_stat, remaining_serv_time, remaining_demand_time, l_print])
    #time to serve less then time to recieve new demand
    if remaining_serv_time <= remaining_demand_time and current_sys_stat > 0:
      if len(table_3) < current_sys_stat+1:
          table_3.append([0, 0, 0, 0])
      if j != 101:
        table_3[current_sys_stat][2] += remaining_serv_time

      t_event = t_event + remaining_serv_time
      current_event_stat = 2
      current_sys_stat -= 1
      task = q.pop(0)
      t_serv = times_of_services[task-1]
      l_print = task
      t_serv_start = t_event - t_serv
      table_2[task - 1][3] = t_serv_start - table_2[task - 1][1]
      if table_2[task - 1][3] < 0.000001:
        table_2[task - 1][3] = 0
      table_2[task - 1][4] = t_serv_start
      table_2[task - 1][5] = t_serv
      table_2[task - 1][6] = t_event

      remaining_demand_time -= remaining_serv_time
      remaining_serv_time = t_serv

    elif remaining_serv_time > remaining_demand_time or current_sys_stat == 0:
      if len(table_3) < current_sys_stat+1:
          table_3.append([0, 0, 0, 0])
      if j != 101:
        table_3[current_sys_stat][2] += remaining_demand_time

      t_event += remaining_demand_time
      current_event_stat = 1
      l_j += 1
      if current_sys_stat == 0:
        remaining_serv_time = times_of_services[l_j-1]
      else:
        remaining_serv_time -= remaining_demand_time
      current_sys_stat += 1
      remaining_demand_time = times_of_demandes[l_j]
      q.append(l_j)
      l_print = l_j
      table_2[l_j - 1] = [l_j, t_event, current_sys_stat-1, -1, -1, -1, -1]

  max_demand_num = 0
  for arr in table_1:
    if arr[-1] > max_demand_num:
      max_demand_num = arr[-1]
  table_2 = table_2[0:max_demand_num]


  for row in table_1:
    table_3[row[3]][0] += 1

  for i in range(len(table_3)):
    table_3[i][1] = table_3[i][0] / 100
    table_3[i][3] = table_3[i][2] / table_1[-1][1]


  return table_1, table_2, table_3

t1, t2, t3 = task3()

for i in range(len(t3)):
    t3[i].insert(0, i)
export_table(t1, "table_31")
export_table(t2, "table_32")
export_table(t3, "table_33")

print(f"Count of demands: {len(t2)}")
print(f"Count of unserved demands: {sum(map((lambda x: x[3] != -1), t2))}")
z = 0
for i in range(100):
    cnt = 0
    t_event = t1[i][1]
    for j in range(len(t2)):
        if t2[j][1] >= t_event:
            cnt += 1
    z += cnt
print(f"z(100): {z/100}")

print(f"Count of demands: {len(t2)}")
serv_demands = sum(map((lambda x: x[2] == 2), t1))
print(f"Count of served demands: {serv_demands}")
z = 0
for i in range(100):
    cnt = 0
    t_event = t1[i][1]
    for j in range(len(t2)):
        if t2[j][1] >= t_event:
            cnt += 1
    z += cnt
print(f"z(100): {z/100}")

sum_q_time = 0
sum_stay_time = 0
for i in range(serv_demands):
    sum_q_time += t2[i][3]
    sum_stay_time += (t2[i][6] - t2[i][1])
q_time_mean = sum_q_time / serv_demands
print(f"q_time_mean: {round(q_time_mean, 5)}")
stay_time_mean = sum_stay_time / serv_demands
print(f"stay_time_mean: {round(stay_time_mean, 5)}")
print(f"downtime: {round(t3[0][3] / t1[-1][1], 5)}")


Count of demands: 61
Count of unserved demands: 40
z(100): 29.4
Count of demands: 61
Count of served demands: 39
z(100): 29.4
q_time_mean: 10.54362
stay_time_mean: 11.62021
downtime: 0.06719


In [6]:
sum_q_time = 0
sum_stay_time = 0
for i in range(serv_demands):
    sum_q_time += t2[i][3]
    sum_stay_time += (t2[i][6] - t2[i][1])
q_time_mean = sum_q_time / serv_demands
print(f"q_time_mean: {round(q_time_mean, 5)}")
stay_time_mean = sum_stay_time / serv_demands
print(f"stay_time_mean: {round(stay_time_mean, 5)}")
print(f"downtime: {round(t3[0][3] / t1[-1][1], 5)}")

q_time_mean: 10.54362
stay_time_mean: 11.62021
downtime: 0.06719
